In [10]:
from llama_index.core import (
    ServiceContext,
    PromptHelper,
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage
)

from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.node_parser import SimpleNodeParser
import tiktoken
# from ..prompts import RecommendationPrompt
from typing import List
import logging

In [11]:
class RAGProvider:

    def __init__(
        self,
        api_key,
        model: str = 'gpt-3.5-turbo',
        persist_dir: str = "tmp"
    ):

        self.model = model
        self.persist_dir = persist_dir

        self.llm = OpenAI(
            model = model,
            temperature = 0,
            max_tokens = 256,
            api_key = api_key
        )

        self.embed_model = OpenAIEmbedding(api_key = api_key)

    def create_index(
        self,
        separator: str = " ",
        chunk_size: int = 1024,
        chunk_overlap: int = 20,
        prompt_context_window: int = 4096,
        num_output: int = 256,
        chunk_overlap_ratio: float = 0.1,
        chunk_size_limit = None,
        input_files: str = "butil/tmp/"
    ):

        node_parser = SimpleNodeParser.from_defaults(
            separator = separator,
            chunk_size = chunk_size,
            chunk_overlap = chunk_overlap,
            tokenizer = tiktoken.encoding_for_model(self.model).encode
        )

        prompt_helper = PromptHelper(
            context_window = prompt_context_window, 
            num_output = num_output, 
            chunk_overlap_ratio = chunk_overlap_ratio, 
            chunk_size_limit = chunk_size_limit
        )

        service_context = ServiceContext.from_defaults(
            llm = self.llm,
            embed_model = self.embed_model,
            node_parser = node_parser,
            prompt_helper = prompt_helper
        )

        documents = SimpleDirectoryReader(input_files).load_data()

        index = VectorStoreIndex.from_documents(
            documents, 
            service_context = service_context
        )

        index.storage_context.persist(persist_dir = self.persist_dir)

In [12]:
import dotenv

In [13]:
dotenv.load_dotenv()

True

In [14]:
import os

In [15]:
# read OPENAI_API_KEY from .env file
api_key = os.getenv("OPENAI_API_KEY")

In [16]:
provider = RAGProvider(api_key = api_key)

In [17]:
provider.create_index()

/var/folders/0j/d0xd373d2tb7zdzcsqrk3mpw0000gn/T/ipykernel_27153/607528180.py:48: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(
